In [1]:
import os
import sys
import time
sys.path.extend(['..'])

import torch
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath} \usepackage{amssymb}')

In [3]:
res_path = '../../results/pretrained'

ece15_file = 'logitsJuan_ECE15_reg_{}.csv' 
ece_file = 'logitsJuan_ECE_reg_{}.csv'
nll_file = 'logitsJuan_NLL_reg_{}.csv'
bri_file = 'logitsJuan_BRI_reg_{}.csv'

In [4]:
ECE15s = [pd.read_csv(os.path.join(res_path, ece15_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]
ECEs = [pd.read_csv(os.path.join(res_path, ece_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]
NLLs = [pd.read_csv(os.path.join(res_path, nll_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]
Bris = [pd.read_csv(os.path.join(res_path, bri_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]

In [5]:
ECE15 = sum(ECE15s)/len(ECE15s)
ECE = sum(ECEs)/len(ECEs)
NLL = sum(NLLs)/len(NLLs)
Bri = sum(Bris)/len(Bris)

In [6]:
def highlight_min(s):
    '''
    highlight the minimum.
    '''
    is_max = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_max]

In [7]:
def highlight_min_br(s):
    print(s['ECE'])
    
    
    s['ECE'].style.apply(highlight_min, subset=s.select_dtypes(float))
    s['NLL'].style.apply(highlight_min, subset=s.select_dtypes(float))
    s['Brier'].style.apply(highlight_min, subset=s.select_dtypes(float))
      
    return s

### CIFAR 10

In [8]:
curr_ece15 = ECE15.loc['cifar10']
curr_ece = ECE.loc['cifar10']
curr_nll = NLL.loc['cifar10']
curr_bri = Bri.loc['cifar10']

In [9]:
curr_ece15[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,2.642584,2.663559,10.686585,4.268243,2.832803,4.561870,1.834526,3.000970
TS,1.014121,0.829599,2.351264,1.198918,0.738372,1.718529,0.516938,0.918689
ETS,1.797012,2.127014,2.328939,1.173737,0.568172,2.881861,1.520533,0.798777
MIR,1.245873,1.154840,0.769365,1.621956,0.966860,1.545232,1.227502,0.884049
BTS,0.803925,0.916503,1.407277,1.277797,0.942596,1.162204,0.821558,1.014570
HTS_torch,1.161412,0.975218,1.265405,1.220092,0.690891,1.424388,0.668687,0.804692
LTS_torch,1.073186,0.902959,1.257671,1.298905,0.818978,1.420101,0.612157,0.922861
HnLTS_torch,1.155872,0.911510,1.479360,1.334129,0.820818,1.485407,0.618029,1.006377
PTS,0.925232,0.676685,1.162005,1.079734,0.704841,1.161124,0.595976,0.826659
PTS_ece,3.212104,1.683981,3.651347,4.221387,3.376767,4.380592,0.804884,3.588991


In [10]:
print(curr_ece15[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex()) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &        TS &       ETS &       MIR &       BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &       PTS &   PTS\_ece \\
Model             &               &           &           &           &           &            &            &              &           &           \\
\midrule
densenet-121      &      2.642584 &  1.014121 &  1.797012 &  1.245873 &  0.803925 &   1.161412 &   1.073186 &     1.155872 &  0.925232 &  3.212104 \\
densenet-169      &      2.663559 &  0.829599 &  2.127014 &  1.154840 &  0.916503 &   0.975218 &   0.902959 &     0.911510 &  0.676685 &  1.683981 \\
resnet-50         &     10.686585 &  2.351264 &  2.328939 &  0.769365 &  1.407277 &   1.265405 &   1.257671 &     1.479360 &  1.162005 &  3.651347 \\
resnet-101        &      4.268243 &  1.198918 &  1.173737 &  1.621956 &  1.277797 &   1.220092 &   1.298905 &     1.334129 &  1.079734 &  4.221387 \\
resnext-29\_8x16   &      2.832803 &  0.738372 &  0.568172 &  0.

In [11]:
curr_ece

,Uncalibrated,TS,ETS,MIR,BTS,PTS,PTS_ece,LTS,HTS,HnLTS,LTS_torch,HTS_torch,HnLTS_torch
Model,,,,,,,,,,,,,
densenet-121,2.835130,1.386325,2.257274,1.425367,1.091841,1.188681,3.298318,1.438661,1.296316,1.349832,1.411550,1.376855,1.352399
densenet-169,2.791077,1.650473,2.309780,1.355161,1.266717,1.039387,2.231041,1.291028,1.294162,1.212882,1.092361,1.034405,1.017144
resnet-50,10.709518,2.544719,2.837071,1.180272,1.864384,1.585177,3.886461,2.524410,2.432137,1.454847,1.428497,1.583076,2.084703
resnet-101,4.450539,1.373523,1.491050,1.927313,1.718641,1.382473,4.408619,1.336975,1.421214,1.696917,1.596087,1.598841,1.404722
resnext-29_8x16,2.883827,1.026017,0.852116,1.092094,1.185582,1.096158,3.462187,1.079328,1.106320,0.927186,1.035076,1.183239,1.142241
vgg-19,4.605082,2.514875,3.304092,1.630511,1.375733,1.397296,4.580389,2.102290,2.152852,1.651571,1.612534,1.735882,1.661927
wide-resnet-28x10,1.929727,0.697262,1.593208,1.268281,1.071903,0.878062,1.191512,0.791909,0.884984,0.727414,0.888355,0.841236,0.868967
wide-resnet-40x10,3.115250,1.041778,1.137043,1.228119,1.275605,1.041734,3.701983,1.069909,1.199265,1.114712,1.105684,1.076345,1.213066


In [12]:
curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,2.835130,2.791077,10.709518,4.450539,2.883827,4.605082,1.929727,3.115250
TS,1.386325,1.650473,2.544719,1.373523,1.026017,2.514875,0.697262,1.041778
ETS,2.257274,2.309780,2.837071,1.491050,0.852116,3.304092,1.593208,1.137043
MIR,1.425367,1.355161,1.180272,1.927313,1.092094,1.630511,1.268281,1.228119
BTS,1.091841,1.266717,1.864384,1.718641,1.185582,1.375733,1.071903,1.275605
HTS_torch,1.376855,1.034405,1.583076,1.598841,1.183239,1.735882,0.841236,1.076345
LTS_torch,1.411550,1.092361,1.428497,1.596087,1.035076,1.612534,0.888355,1.105684
HnLTS_torch,1.352399,1.017144,2.084703,1.404722,1.142241,1.661927,0.868967,1.213066
PTS,1.188681,1.039387,1.585177,1.382473,1.096158,1.397296,0.878062,1.041734
PTS_ece,3.298318,2.231041,3.886461,4.408619,3.462187,4.580389,1.191512,3.701983


In [13]:
print(curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex()) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &        TS &       ETS &       MIR &       BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &       PTS &   PTS\_ece \\
Model             &               &           &           &           &           &            &            &              &           &           \\
\midrule
densenet-121      &      2.835130 &  1.386325 &  2.257274 &  1.425367 &  1.091841 &   1.376855 &   1.411550 &     1.352399 &  1.188681 &  3.298318 \\
densenet-169      &      2.791077 &  1.650473 &  2.309780 &  1.355161 &  1.266717 &   1.034405 &   1.092361 &     1.017144 &  1.039387 &  2.231041 \\
resnet-50         &     10.709518 &  2.544719 &  2.837071 &  1.180272 &  1.864384 &   1.583076 &   1.428497 &     2.084703 &  1.585177 &  3.886461 \\
resnet-101        &      4.450539 &  1.373523 &  1.491050 &  1.927313 &  1.718641 &   1.598841 &   1.596087 &     1.404722 &  1.382473 &  4.408619 \\
resnext-29\_8x16   &      2.883827 &  1.026017 &  0.852116 &  1.

In [14]:
print(curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.2f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &   TS &  ETS &  MIR &  BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &  PTS &  PTS\_ece \\
Model             &               &      &      &      &      &            &            &              &      &          \\
\midrule
densenet-121      &          2.84 & 1.39 & 2.26 & 1.43 & 1.09 &       1.38 &       1.41 &         1.35 & 1.19 &     3.30 \\
densenet-169      &          2.79 & 1.65 & 2.31 & 1.36 & 1.27 &       1.03 &       1.09 &         1.02 & 1.04 &     2.23 \\
resnet-50         &         10.71 & 2.54 & 2.84 & 1.18 & 1.86 &       1.58 &       1.43 &         2.08 & 1.59 &     3.89 \\
resnet-101        &          4.45 & 1.37 & 1.49 & 1.93 & 1.72 &       1.60 &       1.60 &         1.40 & 1.38 &     4.41 \\
resnext-29\_8x16   &          2.88 & 1.03 & 0.85 & 1.09 & 1.19 &       1.18 &       1.04 &         1.14 & 1.10 &     3.46 \\
vgg-19            &          4.61 & 2.51 & 3.30 & 1.63 & 1.38 &       1.74 &       1.61 &      

In [15]:
curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.188129,0.187019,0.789676,0.304690,0.199662,0.299754,0.149671,0.206792
TS,0.161812,0.160774,0.447289,0.216332,0.162096,0.235521,0.136182,0.163392
ETS,0.167319,0.168556,0.451503,0.219902,0.161691,0.242422,0.142409,0.164236
MIR,0.162716,0.160860,0.443231,0.218557,0.165661,0.225514,0.142842,0.165278
BTS,0.183473,0.203507,0.449789,0.241088,0.190863,0.267219,0.175028,0.172891
HTS_torch,0.160669,0.158660,0.438665,0.215263,0.163187,0.230331,0.135608,0.162632
LTS_torch,0.161064,0.158992,0.444692,0.217388,0.162190,0.233028,0.136364,0.163369
HnLTS_torch,0.160418,0.158390,0.438343,0.214465,0.163420,0.229903,0.135504,0.163300
PTS,0.158443,0.154594,0.444093,0.216703,0.162226,0.228817,0.134879,0.160909
PTS_ece,0.262126,0.188517,0.581009,0.367480,0.277030,0.362972,0.139419,0.329517


In [16]:
print(curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.1881 & 0.1618 & 0.1673 & 0.1627 & 0.1835 &     0.1607 &     0.1611 &       0.1604 & 0.1584 &   0.2621 \\
densenet-169      &        0.1870 & 0.1608 & 0.1686 & 0.1609 & 0.2035 &     0.1587 &     0.1590 &       0.1584 & 0.1546 &   0.1885 \\
resnet-50         &        0.7897 & 0.4473 & 0.4515 & 0.4432 & 0.4498 &     0.4387 &     0.4447 &       0.4383 & 0.4441 &   0.5810 \\
resnet-101        &        0.3047 & 0.2163 & 0.2199 & 0.2186 & 0.2411 &     0.2153 &     0.2174 &       0.2145 & 0.2167 &   0.3675 \\
resnext-29\_8x16   &        0.1997 & 0.1621 & 0.1617 & 0.1657 & 0.1909 &     0.1632 &     0.1622 &       0.1634 & 0.1622 &   0.2770 \\
vgg-19            &      

In [17]:
curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.076359,0.075375,0.239194,0.110169,0.082816,0.110094,0.062887,0.081958
TS,0.072888,0.071723,0.203659,0.101096,0.078307,0.101859,0.060798,0.076777
ETS,0.072440,0.071210,0.202883,0.101027,0.078233,0.100686,0.060941,0.076449
MIR,0.072387,0.071438,0.202246,0.101698,0.078487,0.099821,0.061136,0.076852
BTS,0.072232,0.071064,0.203574,0.101902,0.079086,0.100335,0.061111,0.077036
HTS_torch,0.072440,0.070940,0.201083,0.100291,0.078223,0.099873,0.060974,0.076590
LTS_torch,0.072569,0.071170,0.202618,0.101086,0.078369,0.100529,0.060712,0.076711
HnLTS_torch,0.072449,0.070961,0.201302,0.100340,0.078333,0.100022,0.060561,0.076664
PTS,0.072042,0.070633,0.202395,0.100698,0.078049,0.099611,0.060598,0.076442
PTS_ece,0.079458,0.073940,0.213209,0.112028,0.087241,0.111746,0.061237,0.086764


In [18]:
print(curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.0764 & 0.0729 & 0.0724 & 0.0724 & 0.0722 &     0.0724 &     0.0726 &       0.0724 & 0.0720 &   0.0795 \\
densenet-169      &        0.0754 & 0.0717 & 0.0712 & 0.0714 & 0.0711 &     0.0709 &     0.0712 &       0.0710 & 0.0706 &   0.0739 \\
resnet-50         &        0.2392 & 0.2037 & 0.2029 & 0.2022 & 0.2036 &     0.2011 &     0.2026 &       0.2013 & 0.2024 &   0.2132 \\
resnet-101        &        0.1102 & 0.1011 & 0.1010 & 0.1017 & 0.1019 &     0.1003 &     0.1011 &       0.1003 & 0.1007 &   0.1120 \\
resnext-29\_8x16   &        0.0828 & 0.0783 & 0.0782 & 0.0785 & 0.0791 &     0.0782 &     0.0784 &       0.0783 & 0.0780 &   0.0872 \\
vgg-19            &      

### CIFAR 100

In [19]:
curr_ece = ECE.loc['cifar100']
curr_nll = NLL.loc['cifar100']
curr_bri = Bri.loc['cifar100']

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

Metric                     ECE                                          \
Model             Uncalibrated        TS       ETS       MIR       BTS   
Model                                                                    
densenet-121          8.760250  3.933630  3.039655  4.738307  2.768561   
densenet-169          8.932411  3.948774  2.897083  4.996996  3.187847   
resnet-101           11.446499  2.246798  2.243738  3.895105  2.224123   
resnext-29_8x16       9.692425  3.139508  2.675645  4.209867  2.061077   
vgg-19               17.631318  5.133481  5.364392  4.147693  3.885672   
wide-resnet-28x10     5.187933  4.629842  3.546959  3.919492  3.105283   
wide-resnet-40x10    14.784526  4.201723  2.739724  5.343658  3.550641   

Metric                                                               ...  \
Model                   PTS   PTS_ece       LTS       HTS     HnLTS  ...   
Model                                                                ...   
densenet-121       3.220656  3.207186  4.198000  4.092834  3.313878  ...   
densenet-169       3.066816  2.718246  4.229603  4.165437  3.314126  ...   
resnet-101         2.016214  2.468914  2.375905  2.648524  2.268631  ...   
resnext-29_8x16    1.916079  2.232836  3.251355  3.502886  2.021386  ...   
vgg-19             3.363745  3.991610  3.728082  3.578751  3.621421  ...   
wide-resnet-28x10  3.227304  3.793783  4.603869  4.583657  3.574850  ...   
wide-resnet-40x10  3.944552  2.600865  4.425745  4.202580  3.730715  ...   

Metric                Brier                                                    \
Model                   MIR       BTS       PTS   PTS_ece       LTS       HTS   
Model                                                                           
densenet-121       0.304987  0.305248  0.302126  0.302880  0.303898  0.303946   
densenet-169       0.301862  0.301962  0.297920  0.299318  0.299873  0.299931   
resnet-101         0.384248  0.382470  0.380669  0.381789  0.382163  0.381694   
resnext-29_8x16    0.311620  0.309646  0.306999  0.307539  0.309433  0.309963   
vgg-19             0.388874  0.388022  0.384424  0.385496  0.386526  0.388235   
wide-resnet-28x10  0.285417  0.287132  0.282487  0.286203  0.285669  0.285096   
wide-resnet-40x10  0.330068  0.329314  0.324202  0.323051  0.324544  0.324001   

Metric                                                       
Model                 HnLTS LTS_torch HTS_torch HnLTS_torch  
Model                                                        
densenet-121       0.304911  0.304963  0.303184    0.303609  
densenet-169       0.301612  0.301664  0.299293    0.299538  
resnet-101         0.381698  0.381733  0.382068    0.381366  
resnext-29_8x16    0.309001  0.309026  0.308849    0.309673  
vgg-19             0.389501  0.389665  0.386308    0.387832  
wide-resnet-28x10  0.287185  0.287207  0.284323    0.283962  
wide-resnet-40x10  0.328590  0.328706  0.324243    0.323989  

[7 rows x 39 columns]

In [20]:
curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,8.760250,8.932411,11.446499,9.692425,17.631318,5.187933,14.784526
TS,3.933630,3.948774,2.246798,3.139508,5.133481,4.629842,4.201723
ETS,3.039655,2.897083,2.243738,2.675645,5.364392,3.546959,2.739724
MIR,4.738307,4.996996,3.895105,4.209867,4.147693,3.919492,5.343658
BTS,2.768561,3.187847,2.224123,2.061077,3.885672,3.105283,3.550641
HTS_torch,3.480141,3.746898,2.252193,2.738368,3.650830,3.867962,4.004588
LTS_torch,3.256996,3.369869,2.291718,2.115889,3.779596,3.537507,3.778593
HnLTS_torch,3.249018,3.689023,2.334335,2.675162,3.565961,3.696377,3.987642
PTS,3.220656,3.066816,2.016214,1.916079,3.363745,3.227304,3.944552
PTS_ece,3.207186,2.718246,2.468914,2.232836,3.991610,3.793783,2.600865


In [21]:
print(curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.2f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &   TS &  ETS &  MIR &  BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &  PTS &  PTS\_ece \\
Model             &               &      &      &      &      &            &            &              &      &          \\
\midrule
densenet-121      &          8.76 & 3.93 & 3.04 & 4.74 & 2.77 &       3.48 &       3.26 &         3.25 & 3.22 &     3.21 \\
densenet-169      &          8.93 & 3.95 & 2.90 & 5.00 & 3.19 &       3.75 &       3.37 &         3.69 & 3.07 &     2.72 \\
resnet-101        &         11.45 & 2.25 & 2.24 & 3.90 & 2.22 &       2.25 &       2.29 &         2.33 & 2.02 &     2.47 \\
resnext-29\_8x16   &          9.69 & 3.14 & 2.68 & 4.21 & 2.06 &       2.74 &       2.12 &         2.68 & 1.92 &     2.23 \\
vgg-19            &         17.63 & 5.13 & 5.36 & 4.15 & 3.89 &       3.65 &       3.78 &         3.57 & 3.36 &     3.99 \\
wide-resnet-28x10 &          5.19 & 4.63 & 3.55 & 3.92 & 3.11 &       3.87 &       3.54 &      

In [22]:
curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.893868,0.874816,1.134310,0.939844,1.541421,0.817338,1.224773
TS,0.835487,0.815570,1.000677,0.822038,1.199659,0.813459,0.905464
ETS,0.859196,0.832331,1.009053,0.839845,1.206897,0.830835,0.926263
MIR,0.837843,0.816676,1.010458,0.833204,1.186490,0.799622,0.925424
BTS,0.827085,0.808159,1.003960,0.818997,1.192554,0.787617,0.903724
HTS_torch,0.819009,0.801315,1.006663,0.815291,1.175813,0.778474,0.883636
LTS_torch,0.825375,0.806210,1.000225,0.812816,1.194054,0.787080,0.899361
HnLTS_torch,0.820980,0.802341,1.003550,0.818328,1.182101,0.777002,0.883714
PTS,0.813926,0.792173,0.997573,0.805927,1.172123,0.767565,0.880728
PTS_ece,0.830208,0.811436,1.003628,0.811611,1.190501,0.803357,0.892420


In [23]:
print(curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.8939 & 0.8355 & 0.8592 & 0.8378 & 0.8271 &     0.8190 &     0.8254 &       0.8210 & 0.8139 &   0.8302 \\
densenet-169      &        0.8748 & 0.8156 & 0.8323 & 0.8167 & 0.8082 &     0.8013 &     0.8062 &       0.8023 & 0.7922 &   0.8114 \\
resnet-101        &        1.1343 & 1.0007 & 1.0091 & 1.0105 & 1.0040 &     1.0067 &     1.0002 &       1.0036 & 0.9976 &   1.0036 \\
resnext-29\_8x16   &        0.9398 & 0.8220 & 0.8398 & 0.8332 & 0.8190 &     0.8153 &     0.8128 &       0.8183 & 0.8059 &   0.8116 \\
vgg-19            &        1.5414 & 1.1997 & 1.2069 & 1.1865 & 1.1926 &     1.1758 &     1.1941 &       1.1821 & 1.1721 &   1.1905 \\
wide-resnet-28x10 &      

In [24]:
curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.317058,0.314196,0.405327,0.327484,0.443283,0.289225,0.370023
TS,0.304773,0.301670,0.381723,0.309558,0.391811,0.288609,0.327633
ETS,0.304354,0.301037,0.381624,0.309294,0.391021,0.287947,0.326992
MIR,0.304987,0.301862,0.384248,0.311620,0.388874,0.285417,0.330068
BTS,0.305248,0.301962,0.382470,0.309646,0.388022,0.287132,0.329314
HTS_torch,0.303184,0.299293,0.382068,0.308849,0.386308,0.284323,0.324243
LTS_torch,0.304963,0.301664,0.381733,0.309026,0.389665,0.287207,0.328706
HnLTS_torch,0.303609,0.299538,0.381366,0.309673,0.387832,0.283962,0.323989
PTS,0.302126,0.297920,0.380669,0.306999,0.384424,0.282487,0.324202
PTS_ece,0.302880,0.299318,0.381789,0.307539,0.385496,0.286203,0.323051


In [25]:
print(curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.3171 & 0.3048 & 0.3044 & 0.3050 & 0.3052 &     0.3032 &     0.3050 &       0.3036 & 0.3021 &   0.3029 \\
densenet-169      &        0.3142 & 0.3017 & 0.3010 & 0.3019 & 0.3020 &     0.2993 &     0.3017 &       0.2995 & 0.2979 &   0.2993 \\
resnet-101        &        0.4053 & 0.3817 & 0.3816 & 0.3842 & 0.3825 &     0.3821 &     0.3817 &       0.3814 & 0.3807 &   0.3818 \\
resnext-29\_8x16   &        0.3275 & 0.3096 & 0.3093 & 0.3116 & 0.3096 &     0.3088 &     0.3090 &       0.3097 & 0.3070 &   0.3075 \\
vgg-19            &        0.4433 & 0.3918 & 0.3910 & 0.3889 & 0.3880 &     0.3863 &     0.3897 &       0.3878 & 0.3844 &   0.3855 \\
wide-resnet-28x10 &      

# OLD

### Cars

In [26]:
curr_ece = ECE.loc[ECE.Dataset=='cars'].set_index('Model')
curr_nll = NLL.loc[ECE.Dataset=='cars'].set_index('Model')
curr_bri = Bri.loc[ECE.Dataset=='cars'].set_index('Model')

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

AttributeError: 'DataFrame' object has no attribute 'Dataset'

In [ ]:
curr_ece.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

In [ ]:
curr_nll.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

In [ ]:
curr_bri.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

### Birds

In [ ]:
curr_ece = ECE.loc[ECE.Dataset=='birds'].set_index('Model')
curr_nll = NLL.loc[ECE.Dataset=='birds'].set_index('Model')
curr_bri = Bri.loc[ECE.Dataset=='birds'].set_index('Model')

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

In [ ]:
curr_ece.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

In [ ]:
curr_nll.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

In [ ]:
curr_bri.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

### SVHN

In [ ]:
curr_ece = ECE.loc[ECE.Dataset=='svhn'].set_index('Model')
curr_nll = NLL.loc[ECE.Dataset=='svhn'].set_index('Model')
curr_bri = Bri.loc[ECE.Dataset=='svhn'].set_index('Model')

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

In [ ]:
curr_ece.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

In [ ]:
curr_nll.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

In [ ]:
curr_bri.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)